<a href="https://colab.research.google.com/github/gKorada/MLPractice/blob/main/3_Agent_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

In [ ]:
import pandas as pd
import transformers
import torch
from huggingface_hub import login
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, classification_report

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_token')
login(token = hf_token)

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

cpu


In [ ]:
class LLaMASentimentAgent:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

    def analyze_sentiment(self, review, prompt_type):
        if prompt_type == "critic":
            prompt= [
               {'role': 'system', 'content': """
                                  "You are a highly respected movie critic with over 20 years of experience, known for your insightful and balanced reviews.
                                  our task is to analyze the sentiment of the following movie review based on tone, language, and overall message.
                                  Classify the review strictly as 'positive', 'negative', or 'neutral' and provide only the classification as your response.
                                  Do not include explanations.

                            Sentiment:
                                  """},
              {'role': 'user', 'content': f'What are the sentiments of these reviews {[review]}'}
            ]

        elif prompt_type == "viewer":
            prompt = [
          {'role': 'system', 'content': """
                                  You are a avid movie-goer who often goes to see new an dpopular films in the theater.
                  Use your knowledge and experience to analyze the sentiment of the following movie review.
                  Classify the review strictly as 'positive', 'negative', or 'neutral' and provide only the classification as your response.
                  Do not include explanations.
                  Sentiment:
                                  """},
          {'role': 'user', 'content': f'What are the sentiments of these reviews {[review]}'}
          ]
        elif prompt_type == "english":
            prompt = [
                {'role': 'system', 'content': """
                  You are an esteemed english literature professor at a top 10 univeristy in the country.
                  Use your knowledge and experience to analyze the sentiment of the following movie review.
                  Classify the review strictly as 'positive', 'negative', or 'neutral' and provide only the classification as your response.
                  Do not include explanations.
                  Review: {review}
                  Sentiment:
                """           },
              {'role': 'user', 'content': f'What are the sentiments of these reviews {[review]}'}
          ]
        else:
            raise ValueError("Invalid prompt_type. Choose from 'critic', 'viewer', or 'english'.")

        prompt_text = "".join([f"{d['role']}: {d['content']}" for d in prompt])
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = self.model.generate(**inputs, max_length=100)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

        if "positive" in response:
            return "positive"
        elif "negative" in response:
            return "negative"
        else:
            return "neutral"


def aggregate_predictions(predictions):
    sentiment_counts = {"positive": predictions.count("positive"), "negative": predictions.count("negative"), "neutral": predictions.count("neutral")}
    return max(sentiment_counts, key=sentiment_counts.get)


In [ ]:
model_path = "meta-llama/Llama-3.1-8b-Instruct"  # Replace with the correct path

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/movie_reviews (1).csv')

critic_agent = LLaMASentimentAgent(model_path)
viewer_agent = LLaMASentimentAgent(model_path)
english_agent = LLaMASentimentAgent(model_path)
agents = [critic_agent, viewer_agent, english_agent]



df["critic_agent"] = df["review"].apply(lambda x: agents[0].analyze_sentiment(x, 'critic'))
df["viewer_agent"] = df["review"].apply(lambda x: agents[1].analyze_sentiment(x, 'viewer'))
df["english_agent"] = df["review"].apply(lambda x: agents[2].analyze_sentiment(x, 'english'))
df["final_prediction"] = df.apply(lambda row: aggregate_predictions([row["critic_agent"], row["viewer_agent"],
                                                                     row["english_agent"]]), axis=1)

accuracy = accuracy_score(df["sentiment"], df["final_prediction"])
report = classification_report(df["sentiment"], df["final_prediction"])

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).